In [ ]:
# Historical pp curve error calculation: Euclidean distance MSE
#!pipenv install scipy
#!pipenv install fastdtw

In [ ]:
import sys
sys.path.append('../..')
from pymongo import UpdateOne
from pymongo import MongoClient
from tqdm import tqdm
import numpy as np
from exploration.config import mongo_inst
from mlpp.data_collection.sample import osuDumpSampler
import datetime
from datetime import datetime
import pprint
import matplotlib.pyplot as plt
import pickle 
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

In [ ]:
client = MongoClient(port=27017)
db = client.osu_top_db
top_db = mongo_inst["osu_top_db"]
test = osuDumpSampler(top_db)
user_ids = list(map(lambda c: c['_id'], top_db['osu_user_stats'].find({}, {})))

In [ ]:
#DON"T RUN THIS CELL! DATA IS ALREADY SAVED!

#Find users that have more than 50 data points & creates a new list of user ids

# big_user_ids = []
# for i in user_ids:
#     datapts = len(list(db.osu_track_updates.find({"user_id": i}, {"date":1})))
#     if datapts > 50: #arbitrary number
#         big_user_ids.append(i)



In [ ]:
#DON"T RUN THIS CELL! DATA IS ALREADY SAVED!

### Find the date & pp for real and est sorted by date that are within the overlap time periods
## Finds the date & real_pp and then sorts by date

# real_pp = {}
# for i in big_user_ids:
#     real_pp[i] = list(db.osu_track_updates.find( {"user_id": i}, {"_id":0, "timestamp": 1, "pp_raw":1}))
#     real_pp[i].sort(key = lambda x:x["timestamp"])



In [ ]:
#DON"T RUN THIS CELL! DATA IS ALREADY SAVED!

## Finds the date and est_user_pp and then sorts by date

# est_pp = {}
# for i in big_user_ids:
#     est_pp[i] = list(db.osu_scores_high.find( {"user_id": i}, {"_id":0, "date": 1, "mlpp.est_user_pp":1}))
#     est_pp[i].sort(key = lambda x:x["date"]) 



In [ ]:
#DON"T RUN THIS CELL! DATA IS ALREADY SAVED!

## Finds the overlap time periods

# time_comparison = {}
# for i in big_user_ids:
#     earliest_est_date = est_pp[i][0]["date"]
#     earliest_real_date = real_pp[i][0]["timestamp"]
#     latest_est_date = est_pp[i][-1]["date"]
#     latest_real_date = real_pp[i][-1]["timestamp"]
#     if earliest_est_date >= earliest_real_date:
#         start = earliest_est_date
#     else:
#         start = earliest_real_date
#     if latest_est_date <= latest_real_date:
#         end = latest_est_date
#     else:
#         end = latest_real_date
#     time_comparison[i] = {"start": start, "end": end}


#pp=pprint.PrettyPrinter(indent=4)
#pp.pprint(time_comparison)

In [ ]:
#big_user_file = open('big_user_file.obj', 'wb') 
#pickle.dump(big_user_ids, big_user_file)
#real_pp_big_user_file = open('real_pp_big_user_file.obj', 'wb') 
#pickle.dump(real_pp, real_pp_big_user_file)
#est_pp_big_user_file = open('est_pp_big_user_file.obj', 'wb') 
#pickle.dump(est_pp, est_pp_big_user_file)
#overlap_time_big_user_file = open('overlap_time_big_user_file.obj', 'wb') 
overlap_time_big_user_file = open('overlap_time_big_user_file.obj', 'wb') 
pickle.dump(time_comparison, overlap_time_big_user_file)pickle.dump(time_comparison, overlap_time_big_user_file)

In [ ]:
# Run this cell to load all the data
big_user_ids = pickle.load(open("big_user_file.obj", "rb"))
#real_pp = pickle.load(open("real_pp_big_user_file.obj", "rb"))
#est_pp = pickle.load(open("est_pp_big_user_file.obj", "rb"))
time_comparison = pickle.load(open("overlap_time_big_user_file.obj", "rb"))

In [ ]:
bad_users = []
bad_users_index = []
k = 0
for user in time_comparison:
    if time_comparison[user]["start"] >= time_comparison[user]["end"]:
        bad_users.append(user)
        bad_users_index.append(k)
    k = k+1
bad_users_index

In [ ]:
# for i in bad_users_index:
#     big_user_ids = np.delete(big_user_ids, [i])

# for user in bad_users:
#     del real_pp[user]
#     del est_pp[user]
#     del time_comparison[user]

In [ ]:
def CREATE_REAL_USER_TABLE (one_user_id):
    real_start_time = []
    real_pp_points = []
    for i in real_pp[one_user_id]:
        real_start_time.append(i["timestamp"])
        real_pp_points.append(i["pp_raw"])
    real_table = np.column_stack((real_start_time, real_pp_points))
    return real_table

In [ ]:
## Gets the table for real user pp for all big users
real_table_for_all = {}
for user in big_user_ids:
    real_table_for_all[user] = CREATE_REAL_USER_TABLE(user)

In [ ]:
def CREATE_EST_USER_TABLE (one_user_id):
    est_start_time = []
    est_pp_points = []
    for i in est_pp[one_user_id]:
        est_start_time.append(i["date"])
        est_pp_points.append(i["mlpp"]["est_user_pp"])
    est_table = np.column_stack((est_start_time, est_pp_points))
    return est_table

In [ ]:
## Gets the table for est user pp for all big users
est_table_for_all = {}
for user in big_user_ids:
    est_table_for_all[user] = CREATE_EST_USER_TABLE(user)

In [ ]:
def GET_REAL_NPOINTS(n, user):
    real_within = real_table_for_all[user][real_table_for_all[user][:,0] >= time_comparison[user]["start"]]
    real_within = real_within[real_within[:,0] <= time_comparison[user]["end"]]
    real_xp = real_within[:,0]
    real_fp = real_within[:,1]

    begin = real_xp[0] 
    end = real_xp[-1] 
    real_date_list = [] 
    delta = (end - begin)/n
    for i in range(1, n + 1): 
        real_date_list.append((begin+i*delta).timestamp())

    k  = 0
    for i in real_xp:
        real_xp[k] = i.timestamp()
        k+=1

    real_npoints = np.interp(real_date_list,list(real_xp),list(real_fp))
    return real_npoints

In [ ]:
def GET_EST_NPOINTS(n, user):
    est_within = est_table_for_all[user][est_table_for_all[user][:,0] >= time_comparison[user]["start"]]
    est_within = est_within[est_within[:,0] <= time_comparison[user]["end"]]
    xp = est_within[:,0]
    fp = est_within[:,1]

    begin = xp[0] 
    end = xp[-1] 
    date_list = [] 
    delta = (end - begin)/n
    for i in range(1, n + 1): 
        date_list.append((begin+i*delta).timestamp())

    k  = 0
    for i in xp:
        xp[k] = i.timestamp()
        k+=1

    est_npoints = np.interp(date_list,list(xp),list(fp))
    return est_npoints

In [ ]:
## MSE
def GET_MSE(n, user):
    real_points = GET_REAL_NPOINTS(n, user)
    est_points = GET_EST_NPOINTS(n, user)
    mse_for_one = (np.square(real_points - est_points)).mean()
    return mse_for_one

In [ ]:
### Calculate mse for one user
mse_for_all = {}
subset = big_user_ids[850:855]

for user in subset:
    mse_for_all[user] = GET_MSE(50, user)
mse_for_all

In [ ]:
## Difference in area
### Get 10k points of real pp
x = []
real_within = real_table[real_table[:,0] >= time_comparison[first_user]["start"]]
real_within = real_within[real_within[:,0] <= time_comparison[first_user]["end"]]
real_xp = real_within[:,0]
real_fp = real_within[:,1]

begin = real_xp[0] 
end = real_xp[-1] 
intervals = 10000 
real_date_list = [] 
delta = (end - begin)/intervals 
for i in range(1, intervals + 1): 
    real_date_list.append((begin+i*delta).timestamp())

k  = 0
for i in real_xp:
    real_xp[k] = i.timestamp()
    k+=1

real_tenThousand_points = np.interp(real_date_list,list(real_xp),list(real_fp))
#real_tenThousand_points

In [ ]:
### Get 10k points of est pp
x = []
est_within = est_table[est_table[:,0] >= time_comparison[first_user]["start"]]
est_within = est_within[est_within[:,0] <= time_comparison[first_user]["end"]]
xp = est_within[:,0]
fp = est_within[:,1]

begin = xp[0] 
end = xp[-1] 
intervals = 10000 
date_list = [] 
delta = (end - begin)/intervals 
for i in range(1, intervals + 1): 
    date_list.append((begin+i*delta).timestamp())

k  = 0
for i in xp:
    xp[k] = i.timestamp()
    k+=1

est_tenThousand_points = np.interp(date_list,list(xp),list(fp))
#est_tenThousand_points

In [ ]:
### Get area between curve for real and curve for est
a = time_comparison[first_user]["start"].timestamp()
b = time_comparison[first_user]["end"].timestamp()
N = 1000 #arbitrary number
dx = (b - a) / N
x_midpoint = np.linspace(dx / 2, b - dx / 2, N)
total_area_between_curves = 0
for i in range(0, 10000):
    real_midpoint_riemann = real_tenThousand_points[i] * dx
    est_midpoint_riemann = est_tenThousand_points[i] * dx
    area_between_curves_one_point = abs(real_midpoint_riemann - est_midpoint_riemann)
    total_area_between_curves += area_between_curves_one_point
total_area_between_curves = total_area_between_curves / 86400
total_area_between_curves

In [ ]:
## DTW
distance, path = fastdtw(real_within[:,1], est_within[:,1], dist=euclidean)
print(distance)

In [ ]:
# Compare the real and est user pp within the overlap timeframe for the first user
## Visually compare the real and est user pp within the overlap timeframe
# plt.plot(real_table[:,0], real_table[:,1], label="real")
# plt.plot(est_table[:,0], est_table[:,1], label="est")
# plt.xlim(time_comparison[first_user]["start"], time_comparison[first_user]["end"])
# plt.ylim(5500,7500)
# plt.legend()
# plt.title('Real vs ets usesr pp within the overlap timeframe');